In [2]:
import pandas as pd
import numpy as np
# 读取数据
train_df = pd.read_csv('train_dic.csv')
test_df = pd.read_csv('test_dic.csv')

In [3]:
train_df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'product_unique_id',
       'name', 'L1_category_name_en', 'L2_category_name_en',
       'L3_category_name_en', 'L4_category_name_en', 'year', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'is_discount', 'max_discount'],
      dtype='object')

In [4]:
train_df['warehouse']

warehouse
Prague_1       780566
Prague_3       779655
Prague_2       770709
Brno_1         643637
Budapest_1     574582
Munich_1       259287
Frankfurt_1    198931
Name: count, dtype: int64

In [ ]:
# 定义需要处理的列
columns_to_process = ['name','warehouse', 'holiday_name', 'L1_category_name_en', 'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']

# 对每一列进行处理
for column in columns_to_process:
    # 在 train 数据上按列分组，计算 sales 的均值
    mean_sales = train_df.groupby(column)['sales'].mean().reset_index()
    mean_sales.rename(columns={'sales': f'{column}_mean'}, inplace=True)
    
    # 对均值进行log平滑处理
    mean_sales[f'{column}_mean'] = np.log1p(mean_sales[f'{column}_mean'])  # log1p是 log(x+1) 防止log(0)
   
    # 将均值合并到 train 数据中
    train_df = train_df.merge(mean_sales, on=column, how='left')
    
    # 将均值合并到 test 数据中
    test_df = test_df.merge(mean_sales, on=column, how='left')
    if column not in  ['warehouse','L1_category_name_en']:
        # 删除原始列（如果需要）
        train_df.drop(columns=[column], inplace=True)
        test_df.drop(columns=[column], inplace=True)
